In [1]:
#without open AI
import cv2
import numpy as np
import paho.mqtt.client as mqtt
import tkinter as tk
from tkinter import messagebox

# MQTT Configuration
MQTT_BROKER = "mqtt.eclipseprojects.io"
MQTT_PORT = 1883
MQTT_TOPIC = "road/safety/alerts"

# Function to publish MQTT messages
def publish_alert(message):
    client = mqtt.Client()
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
    client.publish(MQTT_TOPIC, message)
    client.disconnect()

# Function to show alert box
def show_alert(message):
    root = tk.Tk()
    root.withdraw()  # Hide main window
    messagebox.showwarning("Threat Alert!", message)
    root.destroy()

# Load YOLO model for person and threat detection
yolo_net = cv2.dnn.readNet("yolo-coco/yolov4.weights", "yolo-coco/yolov4.cfg")
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]

# Load COCO class labels
with open("yolo-coco/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load gender classification model
gender_net = cv2.dnn.readNetFromCaffe(
    "gender-class/gender_deploy.prototxt", "gender-class/gender_net.caffemodel"
)
gender_list = ["Male", "Female"]

# Initialize webcam
cap = cv2.VideoCapture(0)
cv2.namedWindow("Real-time Detection", cv2.WINDOW_NORMAL)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    detections = yolo_net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []
    male_count, female_count = 0, 0
    threats_detected = []

    for detection in detections:
        for object_detected in detection:
            scores = object_detected[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.4:
                center_x, center_y = int(object_detected[0] * width), int(object_detected[1] * height)
                w, h = int(object_detected[2] * width), int(object_detected[3] * height)
                x, y = int(center_x - w / 2), int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    if len(indexes) > 0:
        for i in indexes.flatten():
            class_id, x, y, w, h = class_ids[i], *boxes[i]
            label = classes[class_id]
            roi = frame[y:y + h, x:x + w]

            if label == "person" and roi.size > 0:
                blob_gender = cv2.dnn.blobFromImage(roi, 1, (227, 227), (78.426337, 87.768914, 114.895847), swapRB=False)
                gender_net.setInput(blob_gender)
                gender_preds = gender_net.forward()
                gender = gender_list[np.argmax(gender_preds)]

                if gender == "Male":
                    male_count += 1
                    color = (255, 0, 0)  # Blue for male
                else:
                    female_count += 1
                    color = (0, 255, 0)  # Green for female

                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, gender, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            if label in ["knife", "gun", "scissors"]:
                threats_detected.append(label)
                color = (0, 0, 255)  # Red for threats
                
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, f"Threat: {label}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                
                publish_alert(f"ALERT: Threat detected - {label}")
                show_alert(f"Threat detected")

    cv2.putText(frame, f"Male: {male_count}, Female: {female_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    if male_count > female_count and female_count > 0:
        publish_alert("RED ALERT: Unbalanced male-to-female ratio detected!")
        show_alert("RED ALERT: Unbalanced male-to-female ratio detected!")

    cv2.imshow("Real-time Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
import openai

# OpenAI API Key (Replace with your own key)
openai.api_key = "your_openai_api_key"

# Function to generate AI-based explanation
def generate_threat_explanation(threat_label):
    prompt = f"A {threat_label} has been detected in a public area. Explain the possible risks and recommended safety measures."
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are an AI assistant providing security guidance."},
                  {"role": "user", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"]

# Modify the existing threat detection logic
if label in ["knife", "gun", "scissors"]:
    threats_detected.append(label)
    color = (0, 0, 255)  # Red for threats
    
    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
    cv2.putText(frame, f"Threat: {label}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    ai_explanation = generate_threat_explanation(label)
    
    publish_alert(f"ALERT: Threat detected - {label}\n{ai_explanation}")
    show_alert(f"Threat detected: {label}\n{ai_explanation}")


In [13]:
!pip install --upgrade openai


In [ ]:
import sys
print(sys.executable)


c:\Users\ashar\anaconda3\python.exe


In [ ]:
#Using openAI for Desktop
import cv2
import numpy as np
import paho.mqtt.client as mqtt
import tkinter as tk
from tkinter import messagebox
import openai

# OpenAI API Client (Latest Version)
client = openai.OpenAI(api_key="your_openai_api_key")

# MQTT Configuration
MQTT_BROKER = "mqtt.eclipseprojects.io"
MQTT_PORT = 1883
MQTT_TOPIC = "road/safety/alerts"

# Function to publish MQTT messages
def publish_alert(message):
    client = mqtt.Client()
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
    client.publish(MQTT_TOPIC, message)
    client.disconnect()

# Function to show alert box
def show_alert(message):
    root = tk.Tk()
    root.withdraw()  # Hide main window
    messagebox.showwarning("Threat Alert!", message)
    root.destroy()

# Function to analyze threats using OpenAI GPT
def analyze_threat(threats_detected):
    threat_text = f"Detected threats: {', '.join(threats_detected)}. Suggest a safety response."
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a safety advisor."},
            {"role": "user", "content": threat_text}
        ]
    )

    return response.choices[0].message.content  # Updated return format

# Load YOLO model for object detection
yolo_net = cv2.dnn.readNet("yolo-coco/yolov4.weights", "yolo-coco/yolov4.cfg")
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]

# Load COCO class labels
with open("yolo-coco/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load gender classification model
gender_net = cv2.dnn.readNetFromCaffe(
    "gender-class/gender_deploy.prototxt", "gender-class/gender_net.caffemodel"
)
gender_list = ["Male", "Female"]

# Initialize webcam
cap = cv2.VideoCapture(0)
cv2.namedWindow("Real-time Detection", cv2.WINDOW_NORMAL)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    detections = yolo_net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []
    male_count, female_count = 0, 0
    threats_detected = []

    for detection in detections:
        for object_detected in detection:
            scores = object_detected[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.4:
                center_x, center_y = int(object_detected[0] * width), int(object_detected[1] * height)
                w, h = int(object_detected[2] * width), int(object_detected[3] * height)
                x, y = int(center_x - w / 2), int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    if len(indexes) > 0:
        for i in indexes.flatten():
            class_id, x, y, w, h = class_ids[i], *boxes[i]
            label = classes[class_id]  # Ensure label is assigned

            roi = frame[y:y + h, x:x + w]

            # Gender classification
            if label == "person" and roi.size > 0:
                blob_gender = cv2.dnn.blobFromImage(roi, 1, (227, 227), 
                                                    (78.426337, 87.768914, 114.895847), swapRB=False)
                gender_net.setInput(blob_gender)
                gender_preds = gender_net.forward()
                gender = gender_list[np.argmax(gender_preds)]

                color = (255, 0, 0) if gender == "Male" else (0, 255, 0)
                if gender == "Male":
                    male_count += 1
                else:
                    female_count += 1

                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, gender, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Threat detection
            if label in ["knife", "gun", "scissors"]:
                threats_detected.append(label)
                color = (0, 0, 255)  # Red for threats
                
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, f"Threat: {label}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display male and female counts
    cv2.putText(frame, f"Male: {male_count}, Female: {female_count}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Alert if unbalanced gender ratio
    if male_count > female_count:
        alert_msg = "RED ALERT: Unbalanced male-to-female ratio detected!"
        publish_alert(alert_msg)
        show_alert(alert_msg)

    # AI-based threat analysis
    if threats_detected:
        ai_advice = analyze_threat(threats_detected)
        publish_alert(f"ALERT: {ai_advice}")
        show_alert(ai_advice)

    # Show video frame
    cv2.imshow("Real-time Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#Using openAI for Android
import cv2
import numpy as np
import paho.mqtt.client as mqtt
import tkinter as tk
from tkinter import messagebox
import openai

# OpenAI API Key (Replace with your actual key)
openai.api_key = "your_openai_api_key"

# MQTT Configuration
MQTT_BROKER = "mqtt.eclipseprojects.io"
MQTT_PORT = 1883
MQTT_TOPIC = "road/safety/alerts"

# Android camera URL (replace with your phone's IP)
ANDROID_CAM_URL = "http://192.0.0.4:8080/video"

# Function to publish MQTT messages
def publish_alert(message):
    client = mqtt.Client()
    client.connect(MQTT_BROKER, MQTT_PORT, 60)
    client.publish(MQTT_TOPIC, message)
    client.disconnect()

# Function to show alert box
def show_alert(message):
    root = tk.Tk()
    root.withdraw()
    messagebox.showwarning("Threat Alert!", message)
    root.destroy()

# Function to analyze threats using OpenAI GPT
def analyze_threat(threats_detected):
    threat_text = f"Detected threats: {', '.join(threats_detected)}. Suggest a safety response."
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "You are a safety advisor."},
                  {"role": "user", "content": threat_text}]
    )
    return response["choices"][0]["message"]["content"]

# Load YOLO model
yolo_net = cv2.dnn.readNet("yolo-coco/yolov4.weights", "yolo-coco/yolov4.cfg")
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]

# Load COCO class labels
with open("yolo-coco/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Connect to Android camera
cap = cv2.VideoCapture(ANDROID_CAM_URL)
cv2.namedWindow("Real-time Detection", cv2.WINDOW_NORMAL)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to retrieve frame. Check the camera stream URL.")
        break

    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)
    detections = yolo_net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []
    threats_detected = []

    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.4:
                center_x, center_y = int(obj[0] * width), int(obj[1] * height)
                w, h = int(obj[2] * width), int(obj[3] * height)
                x, y = int(center_x - w / 2), int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    if len(indexes) > 0:
        for i in indexes.flatten():
            class_id, x, y, w, h = class_ids[i], *boxes[i]
            label = classes[class_id]

            # Threat detection
            if label in ["knife", "gun", "scissors"]:
                threats_detected.append(label)
                color = (0, 0, 255)  # Red for threats
                
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, f"Threat: {label}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # AI-based threat analysis
    if threats_detected:
        ai_advice = analyze_threat(threats_detected)
        publish_alert(f"ALERT: {ai_advice}")
        show_alert(ai_advice)

    # Show video frame
    cv2.imshow("Real-time Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
